# Rename study sites with names to their RGI ID numbers

In [2]:
import pandas as pd
import geopandas as gpd
import os
import glob
from tqdm.auto import tqdm

In [3]:
# If using Google Colab, mount Google Drive so you can access the files in this folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
study_sites_path = 'drive/MyDrive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/study-sites/'
site_names = sorted(os.listdir(study_sites_path))
site_names_adj = [x for x in site_names if 'RGI' not in x]
for site_name in tqdm(site_names_adj):
    print(site_name)

    site_path = os.path.join(study_sites_path, site_name)

    # load RGI shapefile
    aoi_path = os.path.join(site_path, 'AOIs')
    aoi_fn = glob.glob(os.path.join(aoi_path, '*RGI*.shp'))[0]
    aoi = gpd.read_file(aoi_fn)

    # grab RGI ID from shapefile
    rgi_id = aoi['RGIId'][0]
    print(rgi_id)

    # rename site folder
    site_path_new = site_path.replace(site_name, rgi_id)
    os.rename(site_path, site_path_new)

    # rename all files with site name in filename to the RGI ID
    # AOIs
    aoi_path_new = os.path.join(site_path_new, 'AOIs')
    aoi_fns = glob.glob(os.path.join(aoi_path_new, '*RGI*'))
    for fn in aoi_fns:
        if '_RGI' in fn:
            fn_new = os.path.basename(fn).replace(site_name + '_RGI', rgi_id)
        else:
            fn_new = os.path.basename(fn).replace(site_name, rgi_id)
        print(os.path.basename(fn), os.path.basename(fn_new))
        os.rename(fn, os.path.join(aoi_path_new, fn_new))
    # DEMs
    dem_fns = glob.glob(os.path.join(site_path_new, 'DEMs', '*.tif'))
    for fn in dem_fns:
        if 'USGS' not in fn:
            fn_new = fn.replace(site_name, rgi_id)
            print(os.path.basename(fn), os.path.basename(fn_new))
            os.rename(fn, fn_new)
    # other directories
    fns_list = [glob.glob(os.path.join(site_path_new, 'imagery', 'classified', '*' + site_name + '*')),
                glob.glob(os.path.join(site_path_new, 'imagery', 'snowlines', '*' + site_name + '*')),
                glob.glob(os.path.join(site_path_new, 'figures', '*' + site_name + '*')),
                glob.glob(os.path.join(site_path_new, 'ERA', '*' + site_name + '*')),
                glob.glob(os.path.join(site_path_new, 'ELAs', '*' + site_name + '*'))
                ]
    for lst in fns_list:
        for fn in lst:
          fn_new = fn.replace(site_name, rgi_id)
          # print(fn_new, os.path.basename(fn_new))
          os.rename(fn, fn_new)

    print(' ')

  0%|          | 0/2 [00:00<?, ?it/s]

Hoh
RGI60-02.18763
Hoh_RGI_outline.prj RGI60-02.18763_outline.prj
Hoh_RGI_outline.cpg RGI60-02.18763_outline.cpg
Hoh_RGI_outline.shx RGI60-02.18763_outline.shx
Hoh_RGI_outline.dbf RGI60-02.18763_outline.dbf
Hoh_RGI_outline.shp RGI60-02.18763_outline.shp
Hoh_RGI_outline.qmd RGI60-02.18763_outline.qmd
Hoh_NASADEM_clip.tif RGI60-02.18763_NASADEM_clip.tif
 
SitKusa
RGI60-01.14391
SitKusa_RGI_outline.shx RGI60-01.14391_outline.shx
SitKusa_RGI_outline.prj RGI60-01.14391_outline.prj
SitKusa_RGI_outline.qmd RGI60-01.14391_outline.qmd
SitKusa_RGI_outline.shp RGI60-01.14391_outline.shp
SitKusa_RGI_outline.dbf RGI60-01.14391_outline.dbf
SitKusa_RGI_outline.cpg RGI60-01.14391_outline.cpg
SitKusa_ArcticDEM_clip_geoid.tif RGI60-01.14391_ArcticDEM_clip_geoid.tif
 


## Compile snowline files and check that the `site_name` column matches the file names

In [5]:
study_sites_path = 'drive/MyDrive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/study-sites/'
site_names = sorted(os.listdir(study_sites_path))
# iterate over sites
for site_name in tqdm(site_names[162:]):

    print(site_name)
    site_path = os.path.join(study_sites_path, site_name)

    # load snowline file names
    snowlines_path = os.path.join(site_path, 'imagery', 'snowlines')
    snowlines_fns = sorted(glob.glob(os.path.join(snowlines_path, '*.csv')))
    # iterate over snowlines and compile
    snowlines_df = pd.DataFrame()
    for snowline_fn in snowlines_fns:
        snowline_df = pd.read_csv(snowline_fn)
        snowlines_df = pd.concat([snowlines_df, snowline_df])

    snowlines_df.reset_index(drop=True, inplace=True)

    # change site_name column if it doesn't match the file name
    snowlines_df['site_name'] = site_name

    # save to file
    snowlines_fn = site_name + '_snowlines.csv'
    snowlines_df.to_csv(os.path.join(site_path, snowlines_fn), index=False)

  0%|          | 0/35 [00:00<?, ?it/s]

RGI60-02.12468
RGI60-02.12483
RGI60-02.12684
RGI60-02.12702
RGI60-02.12721
RGI60-02.12722
RGI60-02.12947
RGI60-02.12969
RGI60-02.13130
RGI60-02.13254
RGI60-02.13284
RGI60-02.13285
RGI60-02.13287
RGI60-02.13290
RGI60-02.13685
RGI60-02.14009
RGI60-02.14017
RGI60-02.14256
RGI60-02.14297
RGI60-02.14323
RGI60-02.16664
RGI60-02.16674
RGI60-02.16722
RGI60-02.16808
RGI60-02.16912
RGI60-02.16923
RGI60-02.17023
RGI60-02.17025
RGI60-02.17029
RGI60-02.17736
RGI60-02.17738
RGI60-02.17739
RGI60-02.17741
RGI60-02.18763
RGI60-02.18778
